In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## selenium 설치 & 내 드라이브에 chromedriver 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install chromedriver-autoinstaller

#!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' # (최초 1회)

In [ ]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

## 라이브러리 임포트

In [ ]:
import sys
import time
import pandas as pd
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller  # setup chrome options

## chrome_options 설정

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"
sys.path.insert(0,chrome_path)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off : cloab은 새창을 지원하지않기 때문에 창 없는 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

driver = webdriver.Chrome(options=chrome_options)


##유튜브 채널 내 검색 영상 url(byCSS_SELECTOR)

In [ ]:
# 검색할 채널 & 키워드 설정
channel_name = "@NetflixKorea"
search_keyword = "중증외상센터"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# 유튜브 특정 채널 내 검색 URL 생성
url = f"https://www.youtube.com/{channel_name}/search?query={encoded_keyword}"

# 유튜브 검색 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 페이지 스크롤 다운 (Infinite Scroll)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 하나의 영상 링크 가져오기 (테스트용)
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
titles = [element.text for element in elements if element.text]
if elements:
    test_url = elements[0].get_attribute("href")
    print("테스트 URL:", test_url)
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")

# 모든 영상 제목 및 링크 가져오기
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
results = [element.get_attribute("href") for element in elements if element.get_attribute("href")]

# DataFrame 변환
results_df = pd.DataFrame({'title': titles, 'urls': results})
results_df['video_id'] = results_df['urls'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (현재 디렉토리에 저장됨)
file_name = f"YouTube_Crawling_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/Mypython/"
full_path = save_path + file_name  # 전체 파일 경로 생성
results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장

# ChromeDriver 종료
driver.quit()

print(f"🎉 크롤링 완료! 데이터가 '{file_name}'로 저장되었습니다.")

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

##유튜브 전체 검색 영상 url(byCSS_SELECTOR)

In [ ]:
# 검색할 키워드 설정
search_keyword = "KPOP MV"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# YouTube 전체 검색 URL 생성
url = f"https://www.youtube.com/results?search_query={encoded_keyword}"

# 유튜브 검색 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 페이지 스크롤 다운 (Infinite Scroll)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 하나의 영상 링크 가져오기 (테스트용)
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
titles = [element.text for element in elements if element.text]
if elements:
    test_url = elements[0].get_attribute("href")
    print("테스트 URL:", test_url)
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")

# 모든 영상 제목 및 링크 가져오기
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
results = [element.get_attribute("href") for element in elements if element.get_attribute("href")]

# DataFrame 변환
results_df = pd.DataFrame({'title': titles, 'urls': results})
results_df['video_id'] = results_df['urls'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (현재 디렉토리에 저장됨)
file_name = f"YouTube_Crawling_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/Mypython/"
full_path = save_path + file_name  # 전체 파일 경로 생성
results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장

# ChromeDriver 종료
driver.quit()

print(f"🎉 크롤링 완료! 데이터가 '{file_name}'로 저장되었습니다.")

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

## 📌 XPath vs CSS Selector 비교

### ✅ 속도
- **XPath**: CSS Selector보다 속도가 느림 (트리 탐색 방식)
- **CSS Selector**: XPath보다 빠르게 요소를 찾을 수 있음

### ✅ 트리 탐색 (부모/형제 요소 접근)
- **XPath**: 부모(`..`), 형제(`following-sibling::`, `preceding-sibling::`) 요소 접근 가능
- **CSS Selector**: 부모 요소 접근 불가능, 자식 및 형제 요소 선택 가능 (`>`, `+`, `~` 사용)

### ✅ 문법 직관성
- **XPath**: 문법이 다소 복잡하고 길어질 수 있음
- **CSS Selector**: CSS 스타일링 방식과 유사하여 직관적임

### ✅ 특정 텍스트 검색
- **XPath**: 특정 텍스트 포함 요소 찾기 가능 (`contains(text(),'Hello')`)
- **CSS Selector**: 직접 지원하지 않음 (JavaScript 필요)

### ✅ 속성(attribute) 기반 검색
- **XPath**: `//img[@alt='example']` 형식으로 속성 검색 가능
- **CSS Selector**: `img[alt='example']` 형식으로 속성 검색 가능

### ✅ 유지보수성
- **XPath**: HTML 구조 변경 시 영향받기 쉬움
- **CSS Selector**: HTML 변경에도 상대적으로 유연함


##유튜브 채널 모든 영상 url(by XPATH)

In [ ]:
# 특정 유튜브 채널의 전체 동영상 가져오기
channel_name = "@DisneyPlusKR"
url = f"https://www.youtube.com/{channel_name}/videos"

# 유튜브 채널의 "동영상" 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 하나의 영상 링크 가져오기 (테스트용)
elements = driver.find_elements(By.XPATH, '//a[@id="video-title-link"]')
if elements:
    test_url = elements[0].get_attribute("href")
    print("테스트 URL:", test_url)
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")

# 무한 스크롤 (Infinite Scroll)로 모든 영상 로드
scroll_pause_time = 3  # 스크롤 후 대기 시간
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 최신 XPath를 사용하여 모든 영상의 제목 및 링크 수집
videos = driver.find_elements(By.XPATH, '//a[@id="video-title-link"]')
video_data = []

for video in videos:
    title = video.get_attribute("title")
    url = video.get_attribute("href")
    if title and url:
        video_data.append({"title": title, "url": url})

# 14. DataFrame 변환
results_df = pd.DataFrame(video_data)
results_df['video_id'] = results_df['url'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 15. 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (현재 디렉토리에 저장됨)
file_name = f"youtube_videos_{channel_name}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/Mypython/"
full_path = save_path + file_name  # 전체 파일 경로 생성
results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장

# WebDriver 종료
driver.quit()

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

##유튜브 채널 내 검색 영상 url(by XPATH)

In [ ]:
# 검색할 채널 & 키워드 설정
channel_name = "@newskbs"
search_keyword = "트럼프"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# 유튜브 특정 채널 내 검색 URL 생성
url = f"https://www.youtube.com/{channel_name}/search?query={encoded_keyword}"

# 유튜브 검색 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 하나의 영상 링크 가져오기 (테스트용)
elements = driver.find_elements(By.XPATH, '//ytd-video-renderer//a[@id="video-title"]')

if elements:
    test_url = elements[0].get_attribute("href")
    print("테스트 URL:", test_url)
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")

# 무한 스크롤 (Infinite Scroll)로 모든 영상 로드
scroll_pause_time = 3  # 스크롤 후 대기 시간
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 최신 XPath를 사용하여 모든 영상의 제목 및 링크 수집
videos = driver.find_elements(By.XPATH, '//ytd-video-renderer//a[@id="video-title"]')
video_data = []

for video in videos:
    title = video.get_attribute("title")
    url = video.get_attribute("href")
    if title and url:
        video_data.append({"title": title, "url": url})

# DataFrame 변환
results_df = pd.DataFrame(video_data)
results_df['video_id'] = results_df['url'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (현재 디렉토리에 저장됨)
file_name = f"youtube_videos_{channel_name}_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/Mypython/"
full_path = save_path + file_name  # 전체 파일 경로 생성
results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장

# WebDriver 종료
driver.quit()

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

## (참고) XPATH 찾는 법

In [ ]:
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 유튜브 특정 채널 내 검색 URL 생성
channel_name = "@NetflixKorea"
search_keyword = "중증외상센터"

url = f"https://www.youtube.com/{channel_name}/search?query={search_keyword}"

# 유튜브 검색 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(2)  # 페이지 로드 대기

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# BeautifulSoup을 사용하여 현재 HTML 구조 가져오기
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# XPath 후보 저장 리스트 (우선순위별 정렬)
xpath_priority = []

# <a> 태그 중 동영상 링크를 포함하는 요소 탐색
for a_tag in soup.find_all("a", href=True):
    href = a_tag["href"]
    if "/watch?" in href:  # 유튜브 동영상 URL 확인
        # 해당 <a> 태그의 ID 또는 Class 속성 확인
        tag_id = a_tag.get("id", "")
        tag_class = " ".join(a_tag.get("class", []))  # 여러 개의 class를 하나의 문자열로 합침
        parent = a_tag.find_parent()  # 부모 요소 찾기

        # 1️⃣ ID 기반 XPath (우선순위 1: 가장 안정적)
        if tag_id:
            xpath_priority.append((1, f"//a[@id='{tag_id}']"))

        # 2️⃣ 부모 태그 + ID 기반 XPath (우선순위 2)
        elif parent and parent.get("id"):
            xpath_priority.append((2, f"//{parent.name}[@id='{parent.get('id')}']//a"))

        # 3️⃣ Class 기반 XPath (우선순위 3)
        elif tag_class:
            xpath_priority.append((3, f"//a[contains(@class, '{tag_class.split()[0]}')]"))  # 첫 번째 class만 사용

        # 4️⃣ 부모 태그 기반 XPath (우선순위 4: 최후의 수단)
        elif parent:
            xpath_priority.append((4, f"//{parent.name}//a[@href='{href}']"))

# XPath 우선순위에 따라 정렬
xpath_priority.sort(key=lambda x: x[0])  # 우선순위가 낮은 숫자(1)에 가까울수록 높은 우선순위

# 최적의 XPath 선택 및 출력
if xpath_priority:
    best_xpath = xpath_priority[0][1]  # 우선순위가 가장 높은 XPath 선택
    print(f"'{best_xpath}'")  # XPath 출력 (문자열 형태)
else:
    print("")
